# 3. Продолжаем изучать SciPy

In [ ]:
import numpy as np
import scipy
import matplotlib.pyplot as plt

## 3.1. scipy.stats

https://docs.scipy.org/doc/scipy/reference/stats.html#module-scipy.stats

scipy.stats содержит информацию о различных вероятностных распределениях и дополняет numpy.random.

In [ ]:
import scipy.stats

Сгенерируем выборку из нормального распределения

In [ ]:
a = np.random.normal(size=10000)

Создадим гистограмму получившегося распределения, используя numpy.histogram

In [ ]:
bins = np.arange(-5, 6, 0.1)

In [ ]:
histogram = np.histogram(a, bins=bins, normed=True)[0]

In [ ]:
bins_centers = 0.5 * (bins[1:] + bins[:-1])

Мы получили вот такое распределение

In [ ]:
plt.plot(bins_centers, histogram) 
plt.show()

Мы можем посмотреть теоретическое распределение, используя scipy.stats

In [ ]:
theoretic = scipy.stats.norm.pdf(bins_centers)

In [ ]:
plt.plot(bins_centers, theoretic)
plt.show()

Изобразим на одном графике

In [ ]:
plt.plot(bins_centers, histogram) 
plt.plot(bins_centers, theoretic) 
plt.show()

Как мы видим распределения почти совпадают

Допустим у нас есть два набора данных

In [ ]:
a = np.random.normal(10, 10, size=100)
b = np.random.normal(10, 1, size=100)

И мы хотим проверить, одинаковые у них средние или нет. При генерации мы указали одинаковые средние (и там и там 10).

In [ ]:
print(np.mean(a))
print(np.mean(b))

Но средние получились разные. Это связано с тем, что выборки недостаточно большие и поэтому возможны разные случайные отклонения.

Ведь если мы будем бросать монетку, у нас может выпасть 7 орлов из 10, хотя в среднем орлов должна быть половина.

Вот если мы возьмём большую выборку, то средние будут похожие.

In [ ]:
a = np.random.normal(10, 10, size=100000)
b = np.random.normal(10, 1, size=100000)

print(np.mean(a))
print(np.mean(b))

scipy предлагает разные способы проверки таких гипотез (совпадают средние или нет)

In [ ]:
scipy.stats.ttest_ind(a, b)   

если p-value меньше 0.05, то средние различаются

In [ ]:
a = np.random.normal(10, 10, size=100)
b = np.random.normal(10, 1, size=100)

print(np.mean(a))
print(np.mean(b))

scipy.stats.ttest_ind(a, b)

In [ ]:
a = np.random.normal(10, 10, size=100000)
b = np.random.normal(10, 1, size=100000)

print(np.mean(a))
print(np.mean(b))

scipy.stats.ttest_ind(a, b)

In [ ]:
a = np.random.normal(10, 10, size=100000)
b = np.random.normal(10.1, 1, size=100000)

print(np.mean(a))
print(np.mean(b))

scipy.stats.ttest_ind(a, b)

## 3.2. scipy.ndimage

https://docs.scipy.org/doc/scipy/reference/ndimage.html#module-scipy.ndimage

scipy.ndimage предоставляет методы для работы с изображениями

In [ ]:
import scipy.ndimage

Скачаем стандартную картинку с енотом

In [ ]:
face = scipy.misc.face(gray=True)

In [ ]:
plt.imshow(face, cmap=plt.cm.gray)
plt.show()

Действительно, енот

In [ ]:
new_face = scipy.ndimage.shift(face, (50, 50))
plt.imshow(new_face, cmap=plt.cm.gray)
plt.show()

Появились чёрные полосы. Чтобы такого не было можем поставить наименования стратегии заполнения полос. В данном случае это взятие ближайшего пикселя:

In [ ]:
new_face = scipy.ndimage.shift(face, (50, 50), mode='nearest')
plt.imshow(new_face, cmap=plt.cm.gray)
plt.show()

Продолжим издеваться над бедным животным и теперь повернём его на 30 градусов.

In [ ]:
new_face = scipy.ndimage.rotate(face, 30)
plt.imshow(new_face, cmap=plt.cm.gray)
plt.show()

Или увеличим в 4 раза. (Обратите внимание на значения на осях)

In [ ]:
new_face = scipy.ndimage.zoom(face, 4)
plt.imshow(new_face, cmap=plt.cm.gray)
plt.show()

Представьте, что у нас оказался битый фотоаппарат, который при съёмке давал шумы (или это кадры с плохой видеокамеры при слабом освещении):

In [ ]:
noisy_face = np.copy(face).astype(np.float)
noisy_face += face.std() * 0.5 * np.random.standard_normal(face.shape)

In [ ]:
plt.imshow(noisy_face, cmap=plt.cm.gray)
plt.show()

Казалось бы, нам теперь не насладиться прекрасным видом шикарного животного. Но scipy.ndimage способен помочь и в такой ситуации.

Зная информацию о соседних пикселях, можно скорректировать значение в текущем пикселе.

In [ ]:
blurred_face = scipy.ndimage.gaussian_filter(noisy_face, sigma=3)
plt.imshow(blurred_face, cmap=plt.cm.gray)
plt.show()

Получилось немного размыто

In [ ]:
median_face = scipy.ndimage.median_filter(noisy_face, size=5)
plt.imshow(median_face, cmap=plt.cm.gray)
plt.show()

А вот так уже лучше.

Есть ещё один способ использования этих фильтров: если сначала размазать изображение, а затем посмотреть на разность с изначальной картикой, то мы получим интересный визуальный эффект:

In [ ]:
blurred_face = scipy.ndimage.gaussian_filter(face, sigma=10)
plt.imshow(face - blurred_face, cmap=plt.cm.gray)
plt.show()

### Бинаризация изображений и операции с черно-белым изображением

![title](morpho_mat1.png)

In [ ]:
a = np.zeros((50, 50))
a[10:-10, 10:-10] = 1
a += 0.25 * np.random.standard_normal(a.shape)
mask = a >= 0.5
opened_mask = scipy.ndimage.binary_opening(mask)
closed_mask = scipy.ndimage.binary_closing(opened_mask)

In [ ]:
plt.figure(figsize=(20, 4))
plt.subplot(1, 4, 1)
plt.imshow(a, cmap=plt.cm.gray)
plt.subplot(1, 4, 2)
plt.imshow(mask, cmap=plt.cm.gray)
plt.subplot(1, 4, 3)
plt.imshow(opened_mask, cmap=plt.cm.gray)
plt.subplot(1, 4, 4)
plt.imshow(closed_mask, cmap=plt.cm.gray)
plt.show()

## 3.3. scipy.interpolate

https://docs.scipy.org/doc/scipy/reference/interpolate.html#module-scipy.interpolate

Интерполяция -  в вычислительной математике способ нахождения промежуточных значений величины по имеющемуся дискретному набору известных значений. И scipy.interpolate предоставляет функционал для этого.

In [ ]:
import scipy.interpolate

Загрузим данные. Величины замерялись каждый год в январе, а как менялись в другое время - неизвестно. Распространим измерения на промежуточные точки.

In [ ]:
max_speeds = np.load('max-speeds.npy')

In [ ]:
original_points = list(range(max_speeds.shape[0]))

Исходные точки

In [ ]:
plt.scatter(original_points, max_speeds)
plt.show()

Самое просто - это линейно соединить точки

In [ ]:
interp = scipy.interpolate.interp1d(original_points, max_speeds, kind='linear')

points = np.arange(0, 20.1, 0.1)

plt.scatter(original_points, max_speeds)
plt.plot(points, interp(points))
plt.show()

Но это не самое хорошее решение - у нас имеются изломы.

In [ ]:
interp = scipy.interpolate.interp1d(original_points, max_speeds, kind='cubic')

points = np.arange(0, 20.1, 0.1)

plt.scatter(original_points, max_speeds)
plt.plot(points, interp(points))
plt.show()

Уже лучше, функция стало гладкой. Но если мы попробуем посчитать значение для не внутренней точки, то мы получим исключение

In [ ]:
interp = scipy.interpolate.interp1d(original_points, max_speeds, kind='cubic')

points = np.arange(0, 21.1, 0.1)

plt.scatter(original_points, max_speeds)
plt.plot(points, interp(points))
plt.show()

Поэтому есть другие методы интерполяции, позволяющие выходить за границу имеющихся точек

In [ ]:
interp = scipy.interpolate.PchipInterpolator(np.array(original_points), np.array(max_speeds))

points = np.arange(0, 21.1, 0.1)

plt.scatter(original_points, max_speeds)
plt.plot(points, interp(points))
plt.show()

## 3.4. Комбинирование модулей

### Снова вернёмся к картинкам

Имеется картинка. Нужно обрезать от неё лишнюю часть и найти на ней три типа сущностей по цвету (чёрное это пустоота, тёмно серый это песок, а остальное это стекло).

In [ ]:
picture = plt.imread('MV_HFV_012.jpg')

In [ ]:
plt.figure(figsize=(12, 10))
plt.imshow(picture, cmap=plt.cm.gray)
plt.show()

Сначала обрежем нижнюю часть, так как она нам не нужна

In [ ]:
picture = picture[:-60]

In [ ]:
plt.figure(figsize=(12, 10))
plt.imshow(picture, cmap=plt.cm.gray)
plt.show()

Мы видим, что  на изоюбражении есть шумы и мелкие точки. Поэтоум используем фильтр, чтобы их убрать.

In [ ]:
filtered_picture = scipy.ndimage.median_filter(picture, size=(7,7))

In [ ]:
plt.figure(figsize=(12, 10))
plt.imshow(filtered_picture, cmap=plt.cm.gray)
plt.show()

Построим гистограмму распределения цветов на картинке (изначальной и после фильтра), чтобы понять где ставить границы

In [ ]:
origin_hist = np.histogram(picture, bins=np.arange(256))
filtered_hist = np.histogram(filtered_picture, bins=np.arange(256))

plt.figure(figsize=(12, 6))
plt.plot(np.arange(255), origin_hist[0]) 
plt.plot(np.arange(255), filtered_hist[0]) 
plt.legend(['original', 'filtered'])
plt.show()

Пики стали более явными. Теперь можем назначить пороги.

In [ ]:
void = filtered_picture <= 75
sand = (filtered_picture > 75) & (filtered_picture <= 114)
glass = filtered_picture > 114

In [ ]:
phases = void.astype(np.int) + 2 * glass.astype(np.int) + 3 * sand.astype(np.int)
plt.figure(figsize=(12, 10))
plt.imshow(phases)
plt.show()

Посмотрим на расположение песка

In [ ]:
plt.figure(figsize=(12, 10))
plt.imshow(sand, cmap=plt.cm.gray)
plt.show()

Нужно убрать линии

In [ ]:
sand_op = scipy.ndimage.binary_opening(sand, iterations=2)

In [ ]:
plt.figure(figsize=(12, 10))
plt.imshow(sand_op, cmap=plt.cm.gray)
plt.show()

Остались мелкие пятнышки, которые не убрали наши преобразованием. Но их уже не так много, как до преобразования. Поэтому для каждого пятнынка, мы можем вычилить его размер и удалить тех, кто содержить меньше 100 пикселей

In [ ]:
sand_labels, sand_nb = scipy.ndimage.label(sand_op)
sand_areas = np.array(scipy.ndimage.sum(sand_op, sand_labels, np.arange(sand_labels.max()+1)))
mask = sand_areas >= 100
remove_small_sand = mask[sand_labels.ravel()].reshape(sand_labels.shape)

In [ ]:
plt.figure(figsize=(12, 10))
plt.imshow(remove_small_sand, cmap=plt.cm.gray)
plt.show()

Аналогично упрощаем пустоту

In [ ]:
void_op = scipy.ndimage.binary_opening(void, iterations=3)

Стекло это всё, что осталось

In [ ]:
glass = (~void_op) & (~remove_small_sand)

И получаем итоговую картинку

In [ ]:
phases = void_op.astype(np.int) + 2 * glass.astype(np.int) + 3 * remove_small_sand.astype(np.int)
plt.figure(figsize=(12, 10))
plt.imshow(phases)
plt.show()